In [18]:
import pandas as pd
import sqlite3

# 1. Connect to sqlite db (creates file if not exists)
conn = sqlite3.connect("ecommerce.db")

# 2. Load raw CSVs
customers = pd.read_csv("customers_dim.csv")
orders = pd.read_csv("orders_fact.csv")
order_items = pd.read_csv("order_items.csv")
products = pd.read_csv("products_dim.csv")
interactions = pd.read_csv("customer_interactions.csv")
marketing = pd.read_csv("marketing_spend.csv")

# 3. Write them into SQLite tables
customers.to_sql("customers_dim", conn, if_exists="replace", index=False)
orders.to_sql("ordesr_fact", conn, if_exists="replace", index=False)
order_items.to_sql("order_items", conn, if_exists="replace", index=False)
products.to_sql("products_dim", conn, if_exists="replace", index=False)
interactions.to_sql("customer_interactions", conn, if_exists="replace", index=False)
marketing.to_sql("marketing_spend", conn, if_exists="replace", index=False)

# 4. Verify tables exist
print(pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn))


                    name
0            orders_fact
1          customers_dim
2            ordesr_fact
3            order_items
4           products_dim
5  customer_interactions
6        marketing_spend


In [27]:
df = pd.read_sql("SELECT * FROM customers_dim", conn)
print(df.head())


   customer_id signup_date   cohort gender age_group acquisition_channel  \
0            1  2023-08-02  2023-08   Male     18-24            Referral   
1            2  2021-05-16  2021-05   Male     25-34            Referral   
2            3  2023-10-17  2023-10  Other     55-64             Organic   
3            4  2020-03-06  2020-03   Male     18-24          Google Ads   
4            5  2022-10-31  2022-10  Other     18-24               Email   

   zipcode loyalty_status  
0    62874            VIP  
1    93957            New  
2    36535         Lapsed  
3    60352        Regular  
4    13275        Regular  


In [28]:
cursor.execute("DROP VIEW IF EXISTS customer_rfm")

In [ ]:
query = """
CREATE TABLE customer_rfm AS
SELECT
    c.customer_id,
    julianday('2025-01-01') - julianday(MAX(o.order_date)) AS recency_days,
    COUNT(o.order_id) AS frequency,
    SUM(o.order_amount - o.discount_amount) AS monetary,
    AVG(o.order_amount) AS avg_order_value,
    julianday('2025-01-01') - julianday(MIN(o.order_date)) AS tenure_days
FROM customers_dim c
LEFT JOIN orders_fact o ON c.customer_id = o.customer_id
GROUP BY c.customer_id;
"""
conn.execute(query)
conn.commit()

In [32]:
df = pd.read_sql("SELECT * FROM customer_rfm", conn)


In [33]:
print(df.head())

   customer_id  recency_days  frequency  monetary  avg_order_value  \
0            1         389.0          8   8741.29      1098.510000   
1            2         471.0          5   7154.05      1434.656000   
2            3         382.0          2   2879.12      1439.560000   
3            4         426.0         44  91064.20      2076.277273   
4            5         380.0         22  53925.91      2454.183636   

   tenure_days  
0        509.0  
1       1078.0  
2        384.0  
3       1721.0  
4        785.0  


In [35]:
cursor.execute("DROP VIEW IF EXISTS customer_lifetime_value")

In [36]:
query = """
CREATE TABLE customer_lifetime_value AS
SELECT 
    c.customer_id,
    SUM(o.order_amount - o.discount_amount) AS total_revenue,
    SUM((oi.quantity * oi.price) - o.discount_amount) AS total_profit
FROM customers_dim c
LEFT JOIN orders_fact o ON c.customer_id = o.customer_id
LEFT JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY c.customer_id;
"""
conn.execute(query)
conn.commit() 

In [37]:
cursor.execute("DROP VIEW IF EXISTS segment_profitability")

In [39]:
query = """
CREATE TABLE segment_profitability AS
SELECT 
    c.customer_id,
    c.age_group,
    c.acquisition_channel,
    c.loyalty_status,
    SUM(o.order_amount - o.discount_amount) AS segment_profit
FROM customers_dim c
LEFT JOIN orders_fact o ON c.customer_id = o.customer_id
WHERE o.order_status = 'Delivered'
GROUP BY c.customer_id, c.age_group, c.acquisition_channel, c.loyalty_status;
"""
conn.execute(query)
conn.commit() 

In [41]:
cursor.execute("DROP VIEW IF EXISTS revenue_by_payment")

In [43]:
query = """
CREATE TABLE revenue_by_payment AS
SELECT 
    c.customer_id,
    o.payment_method,
    SUM(o.order_amount - o.discount_amount) AS revenue_payment_method
FROM customers_dim c
JOIN orders_fact o ON c.customer_id = o.customer_id
WHERE o.order_status = 'Delivered'
GROUP BY c.customer_id, o.payment_method;
"""
conn.execute(query)
conn.commit() 

In [44]:
cursor.execute("DROP VIEW IF EXISTS repeat_purchase")

In [46]:
query = """
CREATE TABLE repeat_purchase AS
SELECT 
    c.customer_id,
    CASE WHEN COUNT(o.order_id) > 1 THEN 1 ELSE 0 END AS repeat_buyer
FROM customers_dim c
LEFT JOIN orders_fact o ON c.customer_id = o.customer_id
WHERE o.order_status = 'Delivered'
GROUP BY c.customer_id;
"""
conn.execute(query)
conn.commit() 

In [47]:
cursor.execute("DROP VIEW IF EXISTS customer_engagement")

In [48]:
query = """
CREATE TABLE customer_engagement AS
SELECT 
    c.customer_id,
    COUNT(i.interaction_id) AS total_interactions,
    SUM(CASE WHEN i.type = 'support' THEN 1 ELSE 0 END) AS support_tickets,
    SUM(CASE WHEN i.type = 'return' THEN 1 ELSE 0 END) AS returns,
    SUM(CASE WHEN i.type = 'review' THEN 1 ELSE 0 END) AS reviews
FROM customers_dim c
LEFT JOIN customer_interactions i ON c.customer_id = i.customer_id
GROUP BY c.customer_id;
"""
conn.execute(query)
conn.commit() 

In [49]:
cursor.execute("DROP VIEW IF EXISTS marketing_influence")

In [50]:
query = """
CREATE TABLE marketing_influence AS
SELECT 
    c.customer_id,
    c.acquisition_channel,
    SUM(m.spend) AS total_spend,
    SUM(m.impressions) AS total_impressions,
    SUM(m.clicks) AS total_clicks
FROM customers_dim c
LEFT JOIN marketing_spend m ON c.acquisition_channel = m.channel
GROUP BY c.customer_id, c.acquisition_channel;
"""
conn.execute(query)
conn.commit() 

In [57]:
cursor.execute("DROP VIEW IF EXISTS customer_features")

In [58]:
query = """
CREATE TABLE customer_features AS
SELECT 
    c.customer_id,
    c.signup_date,
    c.cohort,
    c.gender,
    c.age_group,
    c.acquisition_channel,
    c.loyalty_status,

    r.recency_days,
    r.frequency,
    r.monetary,

    clv.total_revenue,
    clv.total_profit,

    sp.segment_profit,
    rev.payment_method,
    rev.revenue_payment_method,

    rp.repeat_buyer,

    ce.total_interactions,
    ce.support_tickets,
    ce.returns,
    ce.reviews,

    mi.total_spend AS marketing_spend,
    mi.total_impressions,
    mi.total_clicks
FROM customers_dim c
LEFT JOIN customer_rfm r ON c.customer_id = r.customer_id
LEFT JOIN customer_lifetime_value clv ON c.customer_id = clv.customer_id
LEFT JOIN segment_profitability sp ON c.customer_id = sp.customer_id
LEFT JOIN revenue_by_payment rev ON c.customer_id = rev.customer_id
LEFT JOIN repeat_purchase rp ON c.customer_id = rp.customer_id
LEFT JOIN customer_engagement ce ON c.customer_id = ce.customer_id
LEFT JOIN marketing_influence mi ON c.customer_id = mi.customer_id;
"""
conn.execute(query)
conn.commit() 

In [59]:
df = pd.read_sql("SELECT * FROM customer_features", conn)

In [60]:
print(df.head())

   customer_id signup_date   cohort gender age_group acquisition_channel  \
0            1  2023-08-02  2023-08   Male     18-24            Referral   
1            1  2023-08-02  2023-08   Male     18-24            Referral   
2            1  2023-08-02  2023-08   Male     18-24            Referral   
3            1  2023-08-02  2023-08   Male     18-24            Referral   
4            1  2023-08-02  2023-08   Male     18-24            Referral   

  loyalty_status  recency_days  frequency  monetary  ...  payment_method  \
0            VIP         389.0          8   8741.29  ...       Apple Pay   
1            VIP         389.0          8   8741.29  ...   Bank Transfer   
2            VIP         389.0          8   8741.29  ...     Credit Card   
3            VIP         389.0          8   8741.29  ...      Google Pay   
4            VIP         389.0          8   8741.29  ...          PayPal   

   revenue_payment_method  repeat_buyer total_interactions  support_tickets  \
0      